# 🚀 Classificação Semi-Supervisionada com Naïve Bayes

Este notebook demonstra como usar o aprendizado semi-supervisionado para melhorar um modelo quando se tem uma quantidade limitada de dados rotulados.

O fluxo de trabalho será:
1.  Preparar os dados rotulados e simular um conjunto de dados não rotulados.
2.  Usar o Naïve Bayes como base para um modelo semi-supervisionado.
3.  Treinar o modelo com ambos os conjuntos de dados.
4.  Avaliar o desempenho final.

In [ ]:
### 1. Preparar os Dados (Rotulados e Não Rotulados)

# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Criar o DataFrame com todos os dados
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'n_palavras_suspeitas': [8, 0, 4, 2, 10, 1, 6, 3, 9, 0],
    'tamanho_email': [500, 120, 300, 250, 800, 150, 400, 200, 600, 100],
    'tem_link': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
    'rotulo': [1, 0, 1, 0, 1, 0, 1, 0, 1, 1]
}
df = pd.DataFrame(data)

# Separar características (X) e rótulo (y)
X_full = df[['n_palavras_suspeitas', 'tamanho_email', 'tem_link']]
y_full = df['rotulo']

# Dividir os dados em conjuntos de treinamento e teste (mesma divisão de antes)
X_treino_full, X_teste, y_treino_full, y_teste = train_test_split(X_full, y_full, test_size=0.3, random_state=42)

# Agora, vamos simular o cenário semi-supervisionado:
# Pegar uma parte do conjunto de treinamento como 'rotulada' e o resto como 'não rotulada'
X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(
    X_treino_full, y_treino_full, test_size=0.5, random_state=42
)

# Criar o conjunto de dados de treinamento semi-supervisionado
# Os dados 'não rotulados' têm seu rótulo substituído por -1 (convensão do sklearn)
y_treino_semisup = np.concatenate([y_labeled, np.full_like(y_unlabeled, -1)])
X_treino_semisup = np.concatenate([X_labeled, X_unlabeled])

print("Etapas de preparação concluídas.")
print(f"Dados Rotulados para Treino: {len(X_labeled)}")
print(f"Dados Não Rotulados para Treino: {len(X_unlabeled)}")
print(f"Dados para Teste: {len(X_teste)}")

Etapas de preparação concluídas.
Dados Rotulados para Treino: 3
Dados Não Rotulados para Treino: 4
Dados para Teste: 3


### 2. Criar e Treinar o Modelo Semi-Supervisionado

Usaremos o `SelfTrainingClassifier` com o `GaussianNB` como nosso modelo base. O processo é o seguinte:
1.  O modelo base (`GaussianNB`) é treinado apenas com os dados rotulados.
2.  Ele faz previsões nos dados não rotulados.
3.  Se a confiança da previsão for alta (acima de um `threshold`), ele adiciona essa amostra ao conjunto de treinamento.
4.  O processo se repete até que todos os dados não rotulados sejam classificados ou o limite seja atingido.

In [ ]:
# Definir o modelo base
modelo_base = GaussianNB()

# Criar o modelo semi-supervisionado
# Usamos um limiar de 0.8 para adicionar dados ao conjunto de treinamento
modelo_semisup = SelfTrainingClassifier(modelo_base, threshold=0.8)

# Treinar o modelo com os dados rotulados E os não rotulados
modelo_semisup.fit(X_treino_semisup, y_treino_semisup)

print("\nModelo semi-supervisionado treinado com sucesso!")


Modelo semi-supervisionado treinado com sucesso!


### 3. Avaliar o Desempenho do Modelo

Agora, vamos fazer as previsões nos dados de teste (que o modelo nunca viu) e avaliar a performance.

In [ ]:
# Fazer previsões nos dados de teste
previsoes_semisup = modelo_semisup.predict(X_teste)

# Avaliar as previsões
acuracia_semisup = accuracy_score(y_teste, previsoes_semisup)
matriz_semisup = confusion_matrix(y_teste, previsoes_semisup)
relatorio_semisup = classification_report(y_teste, previsoes_semisup, target_names=['Não Spam', 'Spam'])

print(f"\nAcurácia do modelo semi-supervisionado: {acuracia_semisup:.2f}")
print("\nMatriz de Confusão (semi-supervisionada):")
print(matriz_semisup)
print("\nRelatório de Classificação (semi-supervisionado):")
print(relatorio_semisup)


Acurácia do modelo semi-supervisionado: 0.33

Matriz de Confusão (semi-supervisionada):
[[0 2]
 [0 1]]

Relatório de Classificação (semi-supervisionado):
              precision    recall  f1-score   support

    Não Spam       0.00      0.00      0.00         2
        Spam       0.33      1.00      0.50         1

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3



/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SelfTrainingClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sa